In [53]:
%pip install requests 
%pip install beautifulsoup4
%pip install urllib3
%pip install glob

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [55]:
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
import glob
import pandas as pd

In [25]:
def download_file(url, filename):
    load_dotenv()
    usr = os.getenv("USERNAME")
    pswd = os.getenv("PASSWORD")
    session = requests.Session()
    session.auth = (usr, pswd)
    response = session.get(url)
    response.raise_for_status()  
    with open(f"{filename}", "wb") as file:
        file.write(response.content)

In [38]:
def get_first_hdf5(url: str) -> str:
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    for link in soup.find_all("a", href=True):
        if link["href"].endswith(".HDF5"):
            return urllib.parse.urljoin(url, link["href"])
    return None

In [52]:
import os
import requests
from bs4 import BeautifulSoup

def get_first_file(year, doy):
    u = (
        f"https://gpm1.gesdisc.eosdis.nasa.gov/opendap/"
        f"GPM_L2/GPM_2AKu.07/{year}/{doy:03d}/contents.html"
    )
    r = requests.get(u)
    if not r.ok:
        return None
    s = BeautifulSoup(r.text, "html.parser")
    for a in s.find_all("a"):
        h = a.get("href", "")
        if h.endswith(".HDF5.dmr.html"):
            return h.replace(".dmr.html", "")
    return None

def download_file(u, f):
    r = requests.get(u)
    r.raise_for_status()
    with open(os.path.join("data", f), "wb") as file:
        file.write(r.content)

b = "https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L2/GPM_2AKu.07"
t = "?dap4.ce=/FS_Longitude[0:1:20][0:1:48];/FS_Latitude[0:1:20][0:1:48];/FS_PRE_flagPrecip[0:1:20][0:1:48];/FS_CSF_typePrecip[0:1:20][0:1:48];/FS_ScanTime_Year[0:1:20];/FS_ScanTime_DayOfYear[0:1:20];/FS_ScanTime_Month[0:1:20];/FS_ScanTime_DayOfMonth[0:1:20]"
os.makedirs("data", exist_ok=True)

for y in [2016, 2017, 2018]:
    for i in range(20):
        f = get_first_file(y, i)
        if f:
            csv_filename = f"{f}.csv"
            file_path = os.path.join("data", csv_filename)
            if os.path.exists(file_path):
                print(f"File {csv_filename} already exists. Skipping download.")
                continue
            u = f"{b}/{y}/{i:03d}/{f}.dap.csv{t}"
            print(f"Downloading {csv_filename}...")
            download_file(u, csv_filename)

In [70]:
os.makedirs("data_combined", exist_ok=True)
def combine_files(save=False):
    file_path = "data/*.csv"

    # Use glob to get all files with the .csv extension
    csv_files = glob.glob(file_path)

    # Combine all CSV files into one DataFrame
    combined = []
    for file in csv_files:
        combined.append(pd.read_csv(file))

    # we concatenate all the dataframes into one because they were kind-of like [ [2000], [2001]. [2002], [2003], [2004] ] and we want to make it [combined]
    combined = pd.concat(combined, ignore_index=True)
    if save:
        combined.to_csv("data_combined/combined_data.csv", index=False)
    return combined

In [71]:
combined = combine_files(True)

In [72]:
combined

,Dataset: 2A.GPM.Ku.V9-20211125.20160101-S002036-E015307.010461.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160102-S010112-E023343.010477.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160103-S000914-E014145.010492.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160104-S004947-E022218.010508.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160105-S013019-E030249.010524.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160106-S003817-E021048.010539.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160107-S011846-E025117.010555.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160108-S002643-E015913.010570.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160109-S010709-E023940.010586.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20160110-S001503-E014734.010601.V07A.HDF5,...,Dataset: 2A.GPM.Ku.V9-20211125.20180110-S011224-E024458.021973.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180111-S002115-E015349.021988.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180112-S010241-E023515.022004.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180113-S001132-E014406.022019.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180114-S005257-E022532.022035.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180115-S000147-E013421.022050.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180116-S004312-E021546.022066.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180117-S012436-E025711.022082.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180118-S003325-E020559.022097.V07A.HDF5,Dataset: 2A.GPM.Ku.V9-20211125.20180119-S011449-E024723.022113.V07A.HDF5
0,98.1482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98.2481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,98.3481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,98.4483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,98.5483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1111.0
5012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import csv

def transform_csv(input_path, output_path):
    """
    Removes the first row of the CSV and transforms the first column into row labels.
    
    Args:
        input_path (str): Path to the input CSV file.
        output_path (str): Path where to save the transformed CSV.
    """
    with open(input_path, newline="") as infile:
        reader = list(csv.reader(infile))
        
        # Step 1: Remove the first row
        data = reader[1:]
        
        # Step 2: Transform — first column becomes row label, rest are values
        transformed = []
        for row in data:
            # Skip empty or malformed rows
            if not row or len(row) < 2:
                continue
            label = row[0]  # e.g. /FS_Longitude[0]
            values = row[1:]  # the series of numbers
            transformed.append([label] + values)
            
    # Step 3: Write transformed data to new CSV
    with open(output_path, "w", newline="") as outfile:
        writer = csv.writer(outfile)
        writer.writerows(transformed)

    print(f"Transformed CSV saved at: {output_path}")

In [78]:
transform_csv("data_combined/combined_data.csv", "data_combined/transformed_combined_data.csv")

Transformed CSV saved to: data_combined/transformed_combined_data.csv
